In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import sentencepiece as spm
import os

In [2]:
# ===== 1. トークナイザ読み込み =====
sp = spm.SentencePieceProcessor()
sp.load("tokenizer.model")


True

In [3]:
# ===== 2. データセット定義 =====
class TextDataset(Dataset):
    def __init__(self, text, block_size):
        self.data = torch.tensor(sp.encode(text, out_type=int), dtype=torch.long)
        self.block_size = block_size

    def __len__(self):
        return max(0, len(self.data) - self.block_size)

    def __getitem__(self, idx):
        x = self.data[idx:idx+self.block_size]
        y = self.data[idx+1:idx+self.block_size+1]
        return x, y


In [4]:
# ===== 3. モデル定義（最小GPT風） =====
class MiniGPT(nn.Module):
    def __init__(self, vocab_size, n_embd=128, n_layer=2, n_head=4, block_size=64):
        super().__init__()
        self.tok_emb = nn.Embedding(vocab_size, n_embd)
        self.pos_emb = nn.Embedding(block_size, n_embd)
        self.blocks = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=n_embd, nhead=n_head),
            num_layers=n_layer
        )
        self.ln = nn.LayerNorm(n_embd)
        self.head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx):
        B, T = idx.size()
        tok = self.tok_emb(idx)
        pos = self.pos_emb(torch.arange(T, device=idx.device))
        x = tok + pos
        x = self.blocks(x)
        x = self.ln(x)
        logits = self.head(x)
        return logits


In [5]:
# ===== 4. データ準備 =====
with open("data/input.txt", encoding="utf-8") as f:
    text = f.read()

block_size = 64
dataset = TextDataset(text, block_size)
print(f"len(data): {len(dataset.data)}")
print(f"len(dataset): {len(dataset)}")

dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


len(data): 571
len(dataset): 507


In [6]:
# ===== 5. モデル初期化 =====
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MiniGPT(vocab_size=sp.vocab_size(), block_size=block_size).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)


/home/sh1/deepl/mynovel/venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [7]:
# ===== 6. 学習ループ =====
epochs = 50
for epoch in range(epochs):
    total_loss = 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss = F.cross_entropy(logits.view(-1, sp.vocab_size()), y.view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} | loss: {total_loss/len(dataloader):.4f}")


Epoch 1/50 | loss: 5.5381
Epoch 2/50 | loss: 4.0828
Epoch 3/50 | loss: 2.7771
Epoch 4/50 | loss: 1.8913
Epoch 5/50 | loss: 1.3795
Epoch 6/50 | loss: 1.1226
Epoch 7/50 | loss: 0.9905
Epoch 8/50 | loss: 0.9201
Epoch 9/50 | loss: 0.8780
Epoch 10/50 | loss: 0.8448
Epoch 11/50 | loss: 0.8272
Epoch 12/50 | loss: 0.8110
Epoch 13/50 | loss: 0.7995
Epoch 14/50 | loss: 0.7924
Epoch 15/50 | loss: 0.7826
Epoch 16/50 | loss: 0.7782
Epoch 17/50 | loss: 0.7722
Epoch 18/50 | loss: 0.7691
Epoch 19/50 | loss: 0.7668
Epoch 20/50 | loss: 0.7624
Epoch 21/50 | loss: 0.7576
Epoch 22/50 | loss: 0.7558
Epoch 23/50 | loss: 0.7496
Epoch 24/50 | loss: 0.7494
Epoch 25/50 | loss: 0.7473
Epoch 26/50 | loss: 0.7445
Epoch 27/50 | loss: 0.7463
Epoch 28/50 | loss: 0.7423
Epoch 29/50 | loss: 0.7385
Epoch 30/50 | loss: 0.7394
Epoch 31/50 | loss: 0.7430
Epoch 32/50 | loss: 0.7381
Epoch 33/50 | loss: 0.7362
Epoch 34/50 | loss: 0.7347
Epoch 35/50 | loss: 0.7363
Epoch 36/50 | loss: 0.7354
Epoch 37/50 | loss: 0.7321
Epoch 38/5

In [8]:
# ===== 7. モデル保存 =====
os.makedirs("checkpoints", exist_ok=True)
torch.save(model.state_dict(), "checkpoints/model_final.pth")
print("✅ モデルを checkpoints/model_final.pth に保存しました")


✅ モデルを checkpoints/model_final.pth に保存しました


In [9]:
# ===== 8. モデル再読み込み =====
model.load_state_dict(torch.load("checkpoints/model_final.pth", map_location=device))
model.eval()
print("✅ モデルを再読み込みしました")


✅ モデルを再読み込みしました


In [ ]:
# ===== 9. テキスト生成関数 =====
def generate_text(model, sp, start_text="こんにちは", max_new_tokens=50):
    model.eval()
    ids = sp.encode(start_text, out_type=int)
    x = torch.tensor(ids, dtype=torch.long, device=device)[None, :]
    for _ in range(max_new_tokens):
        logits = model(x)
        next_id = torch.argmax(logits[0, -1]).item()
        x = torch.cat([x, torch.tensor([[next_id]], device=device)], dim=1)
    return sp.decode(x[0].tolist())


# ===== 10. テキスト生成 =====
generated = generate_text(model, sp, "AIは", 40)
print("🧠 生成結果:")
print(generated)

In [15]:
# ===== 9' テキスト生成（アップデートバージョン) ==
import random


def generate_text(model, sp, prompt, max_new_tokens=50, temperature=1.0, top_p=0.9, repetition_penalty=1.2):
    model.eval()
    device = next(model.parameters()).device

    tokens = torch.tensor(sp.encode(prompt), dtype=torch.long).unsqueeze(0).to(device)
    generated = tokens.clone()

    for _ in range(max_new_tokens):
        with torch.no_grad():
            logits = model(tokens)[:, -1, :] / temperature

            # 🔹 繰り返しペナルティ
            for token_id in set(generated[0].tolist()):
                logits[0, token_id] /= repetition_penalty

            # 🔹 softmax（安全版）
            probs = F.softmax(logits, dim=-1)
            probs = torch.nan_to_num(probs, nan=0.0, posinf=0.0, neginf=0.0)

            # 🔹 nucleus (top-p) サンプリング（安全処理付き）
            sorted_probs, sorted_indices = torch.sort(probs, descending=True)
            cumulative_probs = torch.cumsum(sorted_probs, dim=-1)

            # top-p でフィルタリング
            sorted_probs[cumulative_probs > top_p] = 0.0

            # 正規化前にチェック
            sum_probs = sorted_probs.sum()
            if sum_probs == 0 or torch.isnan(sum_probs):
                # fallback: 均一分布にする（安全策）
                sorted_probs = torch.ones_like(sorted_probs) / sorted_probs.numel()
            else:
                sorted_probs /= sum_probs

            # サンプリング
            next_token = torch.multinomial(sorted_probs, 1)
            next_token = sorted_indices[0, next_token.item()].view(1, 1)

            # 生成トークン更新
            tokens = torch.cat([tokens, next_token], dim=1)
            generated = torch.cat([generated, next_token], dim=1)

            # 🔹 終了トークンなら停止
            if next_token.item() == sp.eos_id():
                break

    return sp.decode(generated[0].tolist())

generated = generate_text(model, sp, "AIは", 40)
print("🧠 生成結果:")
print(generated)

🧠 生成結果:
AIは学習用確単来始 夜えよは可次の単語を学習すな学習します デ明nを使うダ新しいです。 簡出 簡in 簡しますいつこなモデルでも、美しい知、
